In [112]:
import sys
import numpy as np


def read_file(pdb_path):
    lines_atom = []
    with open(pdb_path, 'r', encoding="utf-8") as f:
        for line in f:
            line = line.split()
            if (line[0] == 'ATOM' and line[2] == 'CA'):
                lines_atom.append(line)
    return lines_atom

def get_positions(line_atom):
    lines_position = [line[6:9] for line in line_atom]
    lines_position = np.array(lines_position).astype(float)

    # lines_series = [line[1] for line in line_atom]
    # lines_series = np.array(lines_series).astype(int)

    lines_index = [line[5] for line in line_atom]
    lines_index = np.array(lines_index).astype(int)

    return lines_index, lines_position

def cal_internal_contacts(lines_position, a, b, threshold=10):
    num_internal = 0
    for i in range(a, b-1):
        for j in range(i+1, b):
            _distance = np.sqrt(np.sum( (lines_position[i]-lines_position[j])**2 ))
            if (i!=j and _distance<=threshold):
                num_internal+=1
    return num_internal

def cal_external_contacts(lines_position, a, threshold=10):
    num_external = 0
    for i in range(0, a):
        for j in range(a, len(lines_position)):
            _distance = np.sqrt(np.sum( (lines_position[i]-lines_position[j])**2 ))
            if (i!=j and _distance<=threshold):
                num_external+=1
    return num_external



def cal_domak(lines_position):
    scores_domak = []
    max_score = 0
    index = 0

    for i in range(len(lines_position)):
        i = i+1

        int_A = cal_internal_contacts(lines_position, 0, i)
        int_B = cal_internal_contacts(lines_position, i, len(lines_position))
        ext_AB = cal_external_contacts(lines_position, i)
        if ext_AB!=0:
            _score_domak = (int_A/ext_AB) * (int_B/ext_AB)
            scores_domak.append(_score_domak)

            if max_score < _score_domak:
                max_score = _score_domak
                index = i

    return scores_domak, max_score, index


# pdb_path = sys.argv[1]
pdb_path = './2csn.pdb'
line_atom = read_file(pdb_path)
lines_series, lines_position = get_positions(line_atom)
scores_domak, max_score, index = cal_domak(lines_position)

In [117]:
max_score

120.11301939058171

In [115]:
index

82

In [8]:
import sys
import numpy as np


def read_file(pdb_path):
    lines_atom = []
    with open(pdb_path, 'r', encoding="utf-8") as f:
        for line in f:
            line = line.split()
            if (line[0] == 'ATOM' and line[2] == 'CA'):
                lines_atom.append(line)
    return lines_atom

def get_positions(line_atom):
    lines_position = [line[6:9] for line in line_atom]
    lines_position = np.array(lines_position).astype(float)

    lines_index = [line[5] for line in line_atom]
    lines_index = np.array(lines_index).astype(int)

    return lines_index, lines_position

def cal_distance(lines_index, lines_position, threshold=12):
    pair = []
    # all_distance = np.zeros([len(lines_position), len(lines_position)])
    for i in range(len(lines_position)-1):
        for j in range(i,len(lines_position)):
            _distance = np.sqrt(np.sum( (lines_position[i]-lines_position[j])**2 ))
            # all_distance[i][j] = _distance
            if (i!=j and _distance<=threshold):
                pair.append([lines_index[i], lines_index[j]])
    return np.array(pair)

# def main():
#     # pdb_path = sys.argv[1]
#     pdb_path = './2csn.pdb'
#     lines_atom = read_file(pdb_path)
#     lines_index, lines_position = get_positions(lines_atom)
#     pair = cal_distance(lines_index, lines_position, threshold=10)
#     np.savetxt('result.pair',pair, fmt="%d")

#     print('Done, Please check result.pair')


# if __name__ == "__main__":
#     main()

In [9]:
pdb_path = './2csn.pdb'
lines_atom = read_file(pdb_path)
lines_index, lines_position = get_positions(lines_atom)
pair = cal_distance(lines_index, lines_position)
len(pair)

4052

In [17]:
split_value = 0
split_residue_index = 0
i = 0


scores = []
num_score = 0
for current in lines_index:
    int_A = np.sum((pair[:,0]<=current) & (pair[:,1]<=current))
    int_B = np.sum((pair[:,0]>=current) & (pair[:,1]>=current))
    ext_AB = np.sum((pair[:,0]<=current) & (pair[:,1]>=current))

    if ext_AB != 0:
        _score = (int_A/ext_AB) * (int_B/ext_AB)
        if split_value < _score:
            split_value = _score
            split_residue_index = current
            print(num_score)
            num_score += 1
            scores.append(_score)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [12]:
index

88